# 我是说明，请阅读

我们需要的**原始数据**字段：**日期,广告形式,产品,展现量,点击量,点击率,总加购数,转化率,ROI,总订单行,总订单金额,总费用。**

我们建立的**因变量：点击率/转化率，销售额/ROI，点击量/展现量，加购数/点击量，订单数/加购数**

我们的**自变量：广告类型的0-1变量，每日每渠道的投入金额（总费用）**

**目前的数据频次为每天，如果要改数据频次，因变量不能直接加和，因为他们都是比率**

京东数据应该可以直接生成下来，天猫数据需要改代码里面一些变量的名称之后才能相除，或者也可以将天猫数据的column改成京东的样子（第一行原始数据字段）

**京东**

京东所有广告类型：PUSH海投,购物触点,海投计划，京东展位，京东直投，京选店铺，经典海投，快车，首焦海投

分成五类：
展位（京东展位）、触点（购物触点）、海投（PUSH海投，海投计划，经典海投，首焦海投）、快车（快车，京选店铺）、其他（京东直投）

**天猫**

天猫所有广告类型都是分成sheet放的，有四种：超级推荐，直通车，钻展，品销宝

或许需要手动加变量，格式：在每一页加上四列类型变量（是否为超级推荐，是否为直通车，是否为钻展，是否为品销宝；是的话用1，不是用0）

天猫每个sheet的变量不一样，需要分别处理：

超级推荐：时间，展现，点击，点击率，添加购物车量，点击转化率，投资回报率，成交订单量，成交订单金额，消耗

直通车：日期，展现量，点击量，点击率，总购物车数，点击转化率，投入产出比，总成交笔数，总成交金额(分)，花费(分)

钻展：日期，有效展现，有效点击数，点击率，添加购物车量，点击转化率，投资回报率，成交订单量，成交订单金额，消耗

品销宝：日期，展现量，点击量，点击率，宝贝加购数，转化率，回报率，成交笔数，成交金额，消耗

每个sheet都要加上这四个变量：是否为超级推荐，是否为直通车，是否为钻展，是否为品销宝，让该类别全部为1，其他全部为0


In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('京东一级子类数据每日求和汇总.csv',encoding='gb18030')
df2 = df.loc[:,['日期','广告一级类型','展现量','点击量','点击率','总加购数','转化率','ROI','总订单行','总订单金额','总费用']]
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      1674 non-null   object 
 1   广告一级类型  1674 non-null   object 
 2   展现量     1674 non-null   int64  
 3   点击量     1674 non-null   int64  
 4   点击率     1674 non-null   float64
 5   总加购数    1643 non-null   float64
 6   转化率     1674 non-null   object 
 7   ROI     1674 non-null   object 
 8   总订单行    1674 non-null   int64  
 9   总订单金额   1674 non-null   float64
 10  总费用     1464 non-null   float64
dtypes: float64(4), int64(3), object(4)
memory usage: 144.0+ KB
None


In [2]:
df2 = df2.replace('#DIV/0!', np.NaN)
df2.loc[:,['点击率']] = pd.DataFrame(df2.loc[:,['点击率']],dtype = float)
df2.loc[:,['ROI']] = pd.DataFrame(df2.loc[:,['ROI']],dtype = float)
df2.loc[:,['转化率']] = pd.DataFrame(df2.loc[:,['转化率']],dtype = float)
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      1674 non-null   object 
 1   广告一级类型  1674 non-null   object 
 2   展现量     1674 non-null   int64  
 3   点击量     1674 non-null   int64  
 4   点击率     1674 non-null   float64
 5   总加购数    1643 non-null   float64
 6   转化率     1570 non-null   float64
 7   ROI     1461 non-null   float64
 8   总订单行    1674 non-null   int64  
 9   总订单金额   1674 non-null   float64
 10  总费用     1464 non-null   float64
dtypes: float64(6), int64(3), object(2)
memory usage: 144.0+ KB
None


效果衡量方式：

1.利用ROI

i.点击率/转化率

ii.销售额/ROI

2.利用各层转换率：点击量/展现量，加购数/点击量，订单数/加购数

i.PCA

ii.在三个中寻找最小的一个

In [3]:
df2

日期 广告一级类型     展现量    点击量   点击率   总加购数   转化率    ROI  总订单行  \
0     2021-01-01  搜索类广告   16252    322  1.98  128.0  4.66  19.79    15   
1     2021-01-02  搜索类广告   17389    313  1.80   81.0  2.88   6.81     9   
2     2021-01-03  搜索类广告   14525    222  1.53   26.0  1.80   6.93     4   
3     2021-01-04  搜索类广告   83918    961  1.15  173.0  3.95  12.32    38   
4     2021-01-05  搜索类广告   27475    441  1.61  107.0  5.90  15.18    26   
...          ...    ...     ...    ...   ...    ...   ...    ...   ...   
1669   2021/2/24  直投类广告  529320  21828  4.12   30.0  0.00   0.00     0   
1670   2021/2/25  直投类广告  174663   8520  4.88   11.0  0.00   0.00     0   
1671   2021/2/26  直投类广告     221      0  0.00    0.0   NaN    NaN     0   
1672   2021/2/27  直投类广告       4      0  0.00    0.0   NaN    NaN     0   
1673   2021/2/28  直投类广告       1      0  0.00    0.0   NaN    NaN     0   

        总订单金额      总费用  
0     32085.0  1621.44  
1     10101.0  1483.00  
2      5746.0   828.95  
3     73922.0  6002.39  
4     37533.0  2472.78  
...       ...      ...  
1669      0.0  7169.74  
1670      0.0  2282.33  
1671      0.0     0.00  
1672      0.0     0.00  
1673      0.0     0.00  

[1674 rows x 11 columns]

In [4]:
size=len(df2)

In [8]:
#京东广告类型一级子类：展示类、搜索类、京东直投类
#dummy1=展示类（触点、展位、海投）
#dummy2=搜索类（快车、京选店铺）
#dummy3=直投类（京东直投、直投）

dummy1=[]
d1=['展示类广告']
dummy2=[]
d2=['搜索类广告']
dummy3=[]
d3=['直投类广告']

for i in range(size):
    if df2['广告一级类型'][i] in d1:
        dummy1.append(1)
        dummy2.append(0)
        dummy3.append(0)
        
    elif df2['广告一级类型'][i] in d2:
        dummy1.append(0)
        dummy2.append(1)
        dummy3.append(0)
        
    elif df2['广告一级类型'][i] in d3:
        dummy1.append(0)
        dummy2.append(0)
        dummy3.append(1)

dummy_1=pd.DataFrame(dummy1)
dummy_2=pd.DataFrame(dummy2)
dummy_3=pd.DataFrame(dummy3)

df2['是否为展示类广告']=dummy_1
df2['是否为搜索类广告']=dummy_2
df2['是否为直投类广告']=dummy_3

df2

日期 广告一级类型     展现量    点击量   点击率   总加购数   转化率    ROI  总订单行  \
0     2021-01-01  搜索类广告   16252    322  1.98  128.0  4.66  19.79    15   
1     2021-01-02  搜索类广告   17389    313  1.80   81.0  2.88   6.81     9   
2     2021-01-03  搜索类广告   14525    222  1.53   26.0  1.80   6.93     4   
3     2021-01-04  搜索类广告   83918    961  1.15  173.0  3.95  12.32    38   
4     2021-01-05  搜索类广告   27475    441  1.61  107.0  5.90  15.18    26   
...          ...    ...     ...    ...   ...    ...   ...    ...   ...   
1669   2021/2/24  直投类广告  529320  21828  4.12   30.0  0.00   0.00     0   
1670   2021/2/25  直投类广告  174663   8520  4.88   11.0  0.00   0.00     0   
1671   2021/2/26  直投类广告     221      0  0.00    0.0   NaN    NaN     0   
1672   2021/2/27  直投类广告       4      0  0.00    0.0   NaN    NaN     0   
1673   2021/2/28  直投类广告       1      0  0.00    0.0   NaN    NaN     0   

        总订单金额      总费用  是否为展示类广告  是否为搜索类广告  是否为直投类广告  
0     32085.0  1621.44         0         1         0  
1     10101.0  1483.00         0         1         0  
2      5746.0   828.95         0         1         0  
3     73922.0  6002.39         0         1         0  
4     37533.0  2472.78         0         1         0  
...       ...      ...       ...       ...       ...  
1669      0.0  7169.74         0         0         1  
1670      0.0  2282.33         0         0         1  
1671      0.0     0.00         0         0         1  
1672      0.0     0.00         0         0         1  
1673      0.0     0.00         0         0         1  

[1674 rows x 14 columns]

In [9]:
#index1=点击率/转化率
index1=[]
for i in range(size):
    if df2['转化率'][i] !=0:
        index=df2['点击率'][i]/df2['转化率'][i]
    else:
        index=0
    index1.append(index)
index_1=pd.DataFrame(index1)
df2['点击率/转化率']=index_1
df2

日期 广告一级类型     展现量    点击量   点击率   总加购数   转化率    ROI  总订单行  \
0     2021-01-01  搜索类广告   16252    322  1.98  128.0  4.66  19.79    15   
1     2021-01-02  搜索类广告   17389    313  1.80   81.0  2.88   6.81     9   
2     2021-01-03  搜索类广告   14525    222  1.53   26.0  1.80   6.93     4   
3     2021-01-04  搜索类广告   83918    961  1.15  173.0  3.95  12.32    38   
4     2021-01-05  搜索类广告   27475    441  1.61  107.0  5.90  15.18    26   
...          ...    ...     ...    ...   ...    ...   ...    ...   ...   
1669   2021/2/24  直投类广告  529320  21828  4.12   30.0  0.00   0.00     0   
1670   2021/2/25  直投类广告  174663   8520  4.88   11.0  0.00   0.00     0   
1671   2021/2/26  直投类广告     221      0  0.00    0.0   NaN    NaN     0   
1672   2021/2/27  直投类广告       4      0  0.00    0.0   NaN    NaN     0   
1673   2021/2/28  直投类广告       1      0  0.00    0.0   NaN    NaN     0   

        总订单金额      总费用  是否为展示类广告  是否为搜索类广告  是否为直投类广告   点击率/转化率  
0     32085.0  1621.44         0         1         0  0.424893  
1     10101.0  1483.00         0         1         0  0.625000  
2      5746.0   828.95         0         1         0  0.850000  
3     73922.0  6002.39         0         1         0  0.291139  
4     37533.0  2472.78         0         1         0  0.272881  
...       ...      ...       ...       ...       ...       ...  
1669      0.0  7169.74         0         0         1  0.000000  
1670      0.0  2282.33         0         0         1  0.000000  
1671      0.0     0.00         0         0         1       NaN  
1672      0.0     0.00         0         0         1       NaN  
1673      0.0     0.00         0         0         1       NaN  

[1674 rows x 15 columns]

In [10]:
#index2=销售额/ROI
index2=[]
for i in range(size):
    if df2['ROI'][i] !=0:
        index=df2['总订单金额'][i]/df2['ROI'][i]
    else:
        index=0
    index2.append(index)
index_2=pd.DataFrame(index2)
df2['销售额/ROI']=index_2

In [11]:
df2

日期 广告一级类型     展现量    点击量   点击率   总加购数   转化率    ROI  总订单行  \
0     2021-01-01  搜索类广告   16252    322  1.98  128.0  4.66  19.79    15   
1     2021-01-02  搜索类广告   17389    313  1.80   81.0  2.88   6.81     9   
2     2021-01-03  搜索类广告   14525    222  1.53   26.0  1.80   6.93     4   
3     2021-01-04  搜索类广告   83918    961  1.15  173.0  3.95  12.32    38   
4     2021-01-05  搜索类广告   27475    441  1.61  107.0  5.90  15.18    26   
...          ...    ...     ...    ...   ...    ...   ...    ...   ...   
1669   2021/2/24  直投类广告  529320  21828  4.12   30.0  0.00   0.00     0   
1670   2021/2/25  直投类广告  174663   8520  4.88   11.0  0.00   0.00     0   
1671   2021/2/26  直投类广告     221      0  0.00    0.0   NaN    NaN     0   
1672   2021/2/27  直投类广告       4      0  0.00    0.0   NaN    NaN     0   
1673   2021/2/28  直投类广告       1      0  0.00    0.0   NaN    NaN     0   

        总订单金额      总费用  是否为展示类广告  是否为搜索类广告  是否为直投类广告   点击率/转化率      销售额/ROI  
0     32085.0  1621.44         0         1         0  0.424893  1621.273370  
1     10101.0  1483.00         0         1         0  0.625000  1483.259912  
2      5746.0   828.95         0         1         0  0.850000   829.148629  
3     73922.0  6002.39         0         1         0  0.291139  6000.162338  
4     37533.0  2472.78         0         1         0  0.272881  2472.529644  
...       ...      ...       ...       ...       ...       ...          ...  
1669      0.0  7169.74         0         0         1  0.000000     0.000000  
1670      0.0  2282.33         0         0         1  0.000000     0.000000  
1671      0.0     0.00         0         0         1       NaN          NaN  
1672      0.0     0.00         0         0         1       NaN          NaN  
1673      0.0     0.00         0         0         1       NaN          NaN  

[1674 rows x 16 columns]

In [12]:
#index3=点击量/展现量
index3=[]
for i in range(size):
    if df2['展现量'][i] !=0:
        index=df2['点击量'][i]/df2['展现量'][i]
    else:
        index=0
    index3.append(index)
index_3=pd.DataFrame(index3)
df2['点击量/展现量']=index_3
df2

日期 广告一级类型     展现量    点击量   点击率   总加购数   转化率    ROI  总订单行  \
0     2021-01-01  搜索类广告   16252    322  1.98  128.0  4.66  19.79    15   
1     2021-01-02  搜索类广告   17389    313  1.80   81.0  2.88   6.81     9   
2     2021-01-03  搜索类广告   14525    222  1.53   26.0  1.80   6.93     4   
3     2021-01-04  搜索类广告   83918    961  1.15  173.0  3.95  12.32    38   
4     2021-01-05  搜索类广告   27475    441  1.61  107.0  5.90  15.18    26   
...          ...    ...     ...    ...   ...    ...   ...    ...   ...   
1669   2021/2/24  直投类广告  529320  21828  4.12   30.0  0.00   0.00     0   
1670   2021/2/25  直投类广告  174663   8520  4.88   11.0  0.00   0.00     0   
1671   2021/2/26  直投类广告     221      0  0.00    0.0   NaN    NaN     0   
1672   2021/2/27  直投类广告       4      0  0.00    0.0   NaN    NaN     0   
1673   2021/2/28  直投类广告       1      0  0.00    0.0   NaN    NaN     0   

        总订单金额      总费用  是否为展示类广告  是否为搜索类广告  是否为直投类广告   点击率/转化率      销售额/ROI  \
0     32085.0  1621.44         0         1         0  0.424893  1621.273370   
1     10101.0  1483.00         0         1         0  0.625000  1483.259912   
2      5746.0   828.95         0         1         0  0.850000   829.148629   
3     73922.0  6002.39         0         1         0  0.291139  6000.162338   
4     37533.0  2472.78         0         1         0  0.272881  2472.529644   
...       ...      ...       ...       ...       ...       ...          ...   
1669      0.0  7169.74         0         0         1  0.000000     0.000000   
1670      0.0  2282.33         0         0         1  0.000000     0.000000   
1671      0.0     0.00         0         0         1       NaN          NaN   
1672      0.0     0.00         0         0         1       NaN          NaN   
1673      0.0     0.00         0         0         1       NaN          NaN   

       点击量/展现量  
0     0.019813  
1     0.018000  
2     0.015284  
3     0.011452  
4     0.016051  
...        ...  
1669  0.041238  
1670  0.048780  
1671  0.000000  
1672  0.000000  
1673  0.000000  

[1674 rows x 17 columns]

In [13]:
#index4=加购数/点击量
index4=[]
for i in range(size):
    if df2['点击量'][i] !=0:
        index=df2['总加购数'][i]/df2['点击量'][i]
    else:
        index=0
    index4.append(index)
index_4=pd.DataFrame(index4)
df2['加购数/点击量']=index_4
df2

日期 广告一级类型     展现量    点击量   点击率   总加购数   转化率    ROI  总订单行  \
0     2021-01-01  搜索类广告   16252    322  1.98  128.0  4.66  19.79    15   
1     2021-01-02  搜索类广告   17389    313  1.80   81.0  2.88   6.81     9   
2     2021-01-03  搜索类广告   14525    222  1.53   26.0  1.80   6.93     4   
3     2021-01-04  搜索类广告   83918    961  1.15  173.0  3.95  12.32    38   
4     2021-01-05  搜索类广告   27475    441  1.61  107.0  5.90  15.18    26   
...          ...    ...     ...    ...   ...    ...   ...    ...   ...   
1669   2021/2/24  直投类广告  529320  21828  4.12   30.0  0.00   0.00     0   
1670   2021/2/25  直投类广告  174663   8520  4.88   11.0  0.00   0.00     0   
1671   2021/2/26  直投类广告     221      0  0.00    0.0   NaN    NaN     0   
1672   2021/2/27  直投类广告       4      0  0.00    0.0   NaN    NaN     0   
1673   2021/2/28  直投类广告       1      0  0.00    0.0   NaN    NaN     0   

        总订单金额      总费用  是否为展示类广告  是否为搜索类广告  是否为直投类广告   点击率/转化率      销售额/ROI  \
0     32085.0  1621.44         0         1         0  0.424893  1621.273370   
1     10101.0  1483.00         0         1         0  0.625000  1483.259912   
2      5746.0   828.95         0         1         0  0.850000   829.148629   
3     73922.0  6002.39         0         1         0  0.291139  6000.162338   
4     37533.0  2472.78         0         1         0  0.272881  2472.529644   
...       ...      ...       ...       ...       ...       ...          ...   
1669      0.0  7169.74         0         0         1  0.000000     0.000000   
1670      0.0  2282.33         0         0         1  0.000000     0.000000   
1671      0.0     0.00         0         0         1       NaN          NaN   
1672      0.0     0.00         0         0         1       NaN          NaN   
1673      0.0     0.00         0         0         1       NaN          NaN   

       点击量/展现量   加购数/点击量  
0     0.019813  0.397516  
1     0.018000  0.258786  
2     0.015284  0.117117  
3     0.011452  0.180021  
4     0.016051  0.242630  
...        ...       ...  
1669  0.041238  0.001374  
1670  0.048780  0.001291  
1671  0.000000  0.000000  
1672  0.000000  0.000000  
1673  0.000000  0.000000  

[1674 rows x 18 columns]

In [14]:
#index5=订单数/加购数
index5=[]
for i in range(size):
    if df2['总订单行'][i] !=0:
        index=df2['总加购数'][i]/df2['总订单行'][i]
    else:
        index=0
    index5.append(index)
index_5=pd.DataFrame(index5)
df2['订单数/加购数']=index_5
df2

日期 广告一级类型     展现量    点击量   点击率   总加购数   转化率    ROI  总订单行  \
0     2021-01-01  搜索类广告   16252    322  1.98  128.0  4.66  19.79    15   
1     2021-01-02  搜索类广告   17389    313  1.80   81.0  2.88   6.81     9   
2     2021-01-03  搜索类广告   14525    222  1.53   26.0  1.80   6.93     4   
3     2021-01-04  搜索类广告   83918    961  1.15  173.0  3.95  12.32    38   
4     2021-01-05  搜索类广告   27475    441  1.61  107.0  5.90  15.18    26   
...          ...    ...     ...    ...   ...    ...   ...    ...   ...   
1669   2021/2/24  直投类广告  529320  21828  4.12   30.0  0.00   0.00     0   
1670   2021/2/25  直投类广告  174663   8520  4.88   11.0  0.00   0.00     0   
1671   2021/2/26  直投类广告     221      0  0.00    0.0   NaN    NaN     0   
1672   2021/2/27  直投类广告       4      0  0.00    0.0   NaN    NaN     0   
1673   2021/2/28  直投类广告       1      0  0.00    0.0   NaN    NaN     0   

        总订单金额      总费用  是否为展示类广告  是否为搜索类广告  是否为直投类广告   点击率/转化率      销售额/ROI  \
0     32085.0  1621.44         0         1         0  0.424893  1621.273370   
1     10101.0  1483.00         0         1         0  0.625000  1483.259912   
2      5746.0   828.95         0         1         0  0.850000   829.148629   
3     73922.0  6002.39         0         1         0  0.291139  6000.162338   
4     37533.0  2472.78         0         1         0  0.272881  2472.529644   
...       ...      ...       ...       ...       ...       ...          ...   
1669      0.0  7169.74         0         0         1  0.000000     0.000000   
1670      0.0  2282.33         0         0         1  0.000000     0.000000   
1671      0.0     0.00         0         0         1       NaN          NaN   
1672      0.0     0.00         0         0         1       NaN          NaN   
1673      0.0     0.00         0         0         1       NaN          NaN   

       点击量/展现量   加购数/点击量   订单数/加购数  
0     0.019813  0.397516  8.533333  
1     0.018000  0.258786  9.000000  
2     0.015284  0.117117  6.500000  
3     0.011452  0.180021  4.552632  
4     0.016051  0.242630  4.115385  
...        ...       ...       ...  
1669  0.041238  0.001374  0.000000  
1670  0.048780  0.001291  0.000000  
1671  0.000000  0.000000  0.000000  
1672  0.000000  0.000000  0.000000  
1673  0.000000  0.000000  0.000000  

[1674 rows x 19 columns]

In [35]:
#描述性统计

In [16]:
dependent = df2.loc[:,['点击率/转化率','销售额/ROI','点击量/展现量','加购数/点击量','订单数/加购数']]
independent = df2.loc[:,['是否为展示类广告','是否为搜索类广告','是否为直投类广告','总费用']]

In [17]:
dependent.describe()

点击率/转化率       销售额/ROI      点击量/展现量      加购数/点击量      订单数/加购数
count  1570.000000   1461.000000  1674.000000  1646.000000  1673.000000
mean      1.825142   6384.645671     0.020245     0.220213     8.672790
std       9.707049   6505.664357     0.020150     0.210576    14.009114
min       0.000000      0.000000     0.000000     0.000000     0.000000
25%       0.321845   2014.069457     0.009607     0.083384     4.473759
50%       0.576760   4438.770282     0.013988     0.190725     6.375000
75%       0.963882   8362.681469     0.024495     0.328261     9.285714
max     251.000000  56341.946698     0.230769     3.783598   284.765957

In [18]:
independent.describe()

是否为展示类广告     是否为搜索类广告     是否为直投类广告           总费用
count  1674.000000  1674.000000  1674.000000   1464.000000
mean      0.419952     0.405615     0.174432   6462.412316
std       0.493698     0.491157     0.379594   6473.914946
min       0.000000     0.000000     0.000000      0.000000
25%       0.000000     0.000000     0.000000   2076.352500
50%       0.000000     0.000000     0.000000   4526.680000
75%       1.000000     1.000000     0.000000   8368.972500
max       1.000000     1.000000     1.000000  56318.010000

In [19]:
#生成输出文档
import csv
outfile=open('after.csv','w',newline='')
to_write=csv.writer(outfile)
title=['日期','广告一级类型','展现量','点击量', '点击率','总加购数','转化率','ROI','总订单行','总订单金额','总费用',
       '是否为展示类广告','是否为搜索类广告','是否为直投类广告','点击率/转化率','销售额/ROI','点击量/展现量','加购数/点击量','订单数/加购数']
to_write.writerow(title)
for i in range(size):
    row = df2.iloc[i].values.tolist()
    to_write.writerow(row)
outfile.close()

**以下为PCA代码，不需要，但是保留着，万一有用呢**

In [14]:
#df3=df2.iloc[:,13:]
#df3

点击量/展现量   加购数/点击量  订单数/加购数
0      0.004479  0.000000      0.0
1      0.006029  0.125000      0.0
2      0.010406  0.000000      0.0
3      0.012303  0.000000      0.0
4      0.007307  0.071429      0.0
...         ...       ...      ...
33044  0.333333  0.000000      0.0
33045  0.008434  0.016260     12.0
33046  0.000000  0.000000      0.0
33047  0.027506  0.001350      0.0
33048  0.000000  0.000000      0.0

[33049 rows x 3 columns]

In [15]:
#PCA
#df3_T = df3.T
#df3_T

0         1         2         3         4         5         6      \
点击量/展现量  0.004479  0.006029  0.010406  0.012303  0.007307  0.004558  0.008522   
加购数/点击量  0.000000  0.125000  0.000000  0.000000  0.071429  0.800000  0.050000   
订单数/加购数  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  1.000000   

            7        8      9      ...     33039     33040     33041  33042  \
点击量/展现量  0.011516  0.00202    0.0  ...  0.032895  0.011228  0.009384    0.0   
加购数/点击量  0.083333  4.00000    0.0  ...  0.006192  0.000000  0.002225    0.0   
订单数/加购数  0.000000  0.00000    0.0  ...  0.000000  0.000000  0.000000    0.0   

           33043     33044      33045  33046     33047  33048  
点击量/展现量  0.02337  0.333333   0.008434    0.0  0.027506    0.0  
加购数/点击量  0.00000  0.000000   0.016260    0.0  0.001350    0.0  
订单数/加购数  0.00000  0.000000  12.000000    0.0  0.000000    0.0  

[3 rows x 33049 columns]

In [16]:
#from sklearn import decomposition
#pca = decomposition.PCA(n_components=1)
#pca.fit(df3_T)
#print(pca.transform(df3_T))
#print(pca.explained_variance_ratio_)

[[-176.39928398]
 [-170.76988765]
 [ 347.16917162]]
[0.96580412]


In [17]:
#index6=PCA结果
#index6=[]
#for i in range(size):
#    pca=df3['点击量/展现量'][i]*(-176.4)+df3['加购数/点击量'][i]*(-170.8)+df3['订单数/加购数'][i]*(347.2)
#    index6.append(pca)
#index_6=pd.DataFrame(index6)
#df2['pca']=index_6
#df2

日期  广告形式     产品      展现量    点击量   点击率  总加购数  转化率   ROI  总订单行  \
0      2019/5/30  购物触点  SR501    893.0    4.0  0.45   0.0  0.0  0.00   0.0   
1      2019/5/30  购物触点  SR501   1327.0    8.0  0.60   1.0  0.0  0.00   0.0   
2      2019/5/29  购物触点  SR501    961.0   10.0  1.04   0.0  0.0  0.00   0.0   
3      2019/5/29  购物触点  SR501   1463.0   18.0  1.23   0.0  0.0  0.00   0.0   
4      2019/5/28  购物触点  SR501   1916.0   14.0  0.73   1.0  0.0  0.00   0.0   
...          ...   ...    ...      ...    ...   ...   ...  ...   ...   ...   
33044  2019/12/2  京东直投   京东直投      6.0    2.0  0.33   0.0  0.0  0.00   0.0   
33045  2019/12/1  京东直投   京东直投  87498.0  738.0  0.01  12.0  0.0  0.28   1.0   
33046  2019/12/2  京东直投   京东直投     34.0    0.0  0.00   0.0  0.0  0.00   0.0   
33047  2019/12/1  京东直投   京东直投  26940.0  741.0  0.03   1.0  0.0  0.00   0.0   
33048  2019/12/2  京东直投   京东直投      8.0    0.0  0.00   0.0  0.0  0.00   0.0   

       总订单金额  点击率/转化率     销售额/ROI   点击量/展现量   加购数/点击量  订单数/加购数          pca  
0        0.0      0.0    0.000000  0.004479  0.000000      0.0    -0.790146  
1        0.0      0.0    0.000000  0.006029  0.125000      0.0   -22.413451  
2        0.0      0.0    0.000000  0.010406  0.000000      0.0    -1.835588  
3        0.0      0.0    0.000000  0.012303  0.000000      0.0    -2.170335  
4        0.0      0.0    0.000000  0.007307  0.071429      0.0   -13.488935  
...      ...      ...         ...       ...       ...      ...          ...  
33044    0.0      0.0    0.000000  0.333333  0.000000      0.0   -58.800000  
33045   79.0      0.0  282.142857  0.008434  0.016260     12.0  4162.134922  
33046    0.0      0.0    0.000000  0.000000  0.000000      0.0     0.000000  
33047    0.0      0.0    0.000000  0.027506  0.001350      0.0    -5.082482  
33048    0.0      0.0    0.000000  0.000000  0.000000      0.0     0.000000  

[33049 rows x 17 columns]

In [18]:
#df3.describe()

点击量/展现量       加购数/点击量       订单数/加购数
count  33049.000000  33049.000000  33049.000000
mean       0.024409      0.104120      0.633639
std        0.083388      0.625749      2.840181
min        0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000
50%        0.000000      0.000000      0.000000
75%        0.013550      0.000000      0.000000
max        4.000000     59.000000    110.000000

In [ ]:
#选取 点击量/展现量